## Desafio 2


Crie uma função, que dada a entrada de duas imagens, sendo a primeira um template a buscar e a segunda uma imagem a ser buscada, identifique se o template existe, funcionando como um detector de imagens, seja qual for as condições, proporção, rotação e demais distorçoes.

O objetivo é localizar logotipo de uma determinada empresa em outras imagens, para evitar seu uso indevido ou não autorizado.

Burger King Logo

<img src="imagens/bk.png">

Logotipos

<img src="imagens/brand-logos.png">

In [52]:
import cv2
from matplotlib import pyplot as plt
import numpy as np

In [91]:
image_target = cv2.imread("imagens/bliss-video.jpg")
image_target_gray = cv2.cvtColor(image_target, cv2.COLOR_BGR2GRAY)

image_search = cv2.imread("imagens/bliss.jpg")
image_search_gray = cv2.cvtColor(image_search, cv2.COLOR_BGR2GRAY)

cv2.imshow("Imagem de vídeo", image_target)
cv2.waitKey()
cv2.destroyAllWindows()

cv2.imshow("Imagen da embalagem", image_search)
cv2.waitKey()
cv2.destroyAllWindows()

orb_detector = cv2.ORB_create(2000)

sift_detector = cv2.xfeatures2d.SIFT_create()

kps = orb_detector.detect(image_target_gray, None)
kps_target, desc_target = orb_detector.compute(image_target_gray, kps)

kps = orb_detector.detect(image_search_gray, None)
kps_search, desc_search = orb_detector.compute(image_search_gray, kps)

kps_sift_target, desc_sift_target = sift_detector.detectAndCompute(image_target_gray, None)
kps_sift_search, desc_sift_search = sift_detector.detectAndCompute(image_search_gray, None)

image_detected = image_search.copy()
image_detected = cv2.drawKeypoints(image_detected, kps, image_detected, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imshow("Cristo Redentor Discriminant", image_detected)
cv2.waitKey()
cv2.destroyAllWindows()

image_detected = image_target.copy()
image_detected = cv2.drawKeypoints(image_detected, kps, image_detected, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imshow("Cristo Redentor Discriminant", image_detected)
cv2.waitKey()
cv2.destroyAllWindows()

In [92]:
FLANN_INDEX_LSH = 6
index_params= dict(algorithm = FLANN_INDEX_LSH, table_number = 6, key_size = 12, multi_probe_level = 1) 

search_params = dict(checks=100)

flann = cv2.FlannBasedMatcher(index_params, search_params)

matches = flann.knnMatch(desc_target, desc_search, k=2)

# Need to draw only good matches, so create a mask
matchesMask = [[0,0] for i in range(len(matches))]

# ratio test as per Lowe's paper
for i,(m,n) in enumerate(matches):
    if m.distance < 0.7*n.distance:
        matchesMask[i]=[1,0]
        
draw_params = dict(matchColor = (0,255,0), singlePointColor = (255,0,0), matchesMask = matchesMask, flags = 0)
image_detected = cv2.drawMatchesKnn(image_target, kps_target, image_search, kps_search, matches, None, **draw_params)

cv2.imshow("Logo Identificado", image_detected)
cv2.waitKey()
cv2.destroyAllWindows()

In [93]:
def detectar_embalagem(image):
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 3)

    search_params = dict(checks=100)

    flann = cv2.FlannBasedMatcher(index_params, search_params)

    image_target_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    kps_sift_target, desc_sift_target = sift_detector.detectAndCompute(image_target_gray, None)
    matches = flann.knnMatch(desc_sift_target, desc_sift_search, k=2)
        
    good_matches = []
    
    try:
        for (m,n) in matches:
            if m.distance < 0.7*n.distance:
                good_matches.append(m)
    except:
        pass
    
    return len(good_matches)

In [94]:
def detectar_embalagem_orb(image):
    FLANN_INDEX_LSH = 6
    index_params= dict(algorithm = FLANN_INDEX_LSH, table_number = 6, key_size = 12, multi_probe_level = 1) 

    search_params = dict(checks=100)

    flann = cv2.FlannBasedMatcher(index_params, search_params)

    image_target_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    kps = orb_detector.detect(image_target_gray, None)
    kps_target, desc_target = orb_detector.compute(image_target_gray, kps)

    matches = flann.knnMatch(desc_target, desc_search, k=2)
        
    good_matches = []
    
    try:
        for (m,n) in matches:
            if m.distance < 0.7*n.distance:
                good_matches.append(m)
    except:
        pass
    
    return len(good_matches)

In [97]:
import time

cam_capture = cv2.VideoCapture("video/bliss.avi")

while True:
    ret, image_frame = cam_capture.read()
    
    if ret:
        matches = detectar_embalagem_orb(image_frame)
    
        if matches > 9:
            cv2.putText(image_frame, "Identificado " + str(matches), (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 6)

    
        cv2.imshow("Sketcher ROI", image_frame)

        if cv2.waitKey(1) == 13:
            break
    else:
        break
    time.sleep(0.01)
        
cam_capture.release()
cv2.destroyAllWindows()

In [104]:
cam_capture.release()
cam_capture = cv2.VideoCapture(0)
cv2.destroyAllWindows()

upper_left = (50, 50)
bottom_right = (300, 300)

while True:
    ret, image_frame = cam_capture.read()
    
    if ret:
    
        #Rectangle marker
        r = cv2.rectangle(image_frame, upper_left, bottom_right, (100, 50, 200), 5)
        rect_img = image_frame[upper_left[1] : bottom_right[1], upper_left[0] : bottom_right[0]]
        
        matches = detectar_embalagem_orb(rect_img)
    
        #if matches > 9:
        cv2.putText(image_frame, "Identificado " + str(matches), (50,350), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 6)

        cv2.imshow("Detector de Embalagem", image_frame)
        if cv2.waitKey(1) == 13:
            break
    else:
        break
        
cam_capture.release()
cv2.destroyAllWindows()